### PyImageJ Testing 
Testing PyImageJ's capabilities to do particle image processing

In [3]:
import scyjava as sj              # {optional} Set memory pool
from skimage import io, color            # ImageJ opener, open the image on the background ImageJ
from IPython.display import Image # Display only images
import os                         # Used to manipulate and create directory with python. 
from skimage import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import imagej
from imagej import Mode

In [5]:
# Initialize ImageJ
ij = imagej.init('net.imagej:imagej+net.imagej:imagej-legacy', mode=Mode.HEADLESS)

# Open the image
image_path = '/workspaces/PyImageJ-Particle-Analysis/Particle Images/Ti64_Lot232-EZ2316_1Use_10X_Scale.png'
dataset1 = ij.io().open(image_path)

# Convert to 8-bit grayscale
grayscale = ij.op().run("convert.uint8", dataset1)

# Apply Otsu threshold
thresholded = ij.op().threshold().otsu(grayscale)

# Convert thresholded image to binary
binary_mask = ij.op().convert().bit(thresholded)

# Convert binary mask to a NumPy array
binary_array = ij.py.from_java(binary_mask)
binary_array = np.array(binary_array, dtype=np.uint8)  # Ensure the array is in 8-bit format

# Convert the NumPy array back to ImageJ image
binary_mask_8bit = ij.py.to_java(binary_array)

# Save the binary image temporarily
binary_image_path = '/workspaces/PyImageJ-Particle-Analysis/Particle Images/Binary.tif'
ij.io().save(binary_mask_8bit, binary_image_path)

# Extract the min and max threshold values from the binary mask
min_val = np.min(binary_array)
max_val = np.max(binary_array)

# Prepare macro code with setThreshold and return the result as a string
macro_code = f"""
// Open the binary image
open("{binary_image_path}");

// Set the image to 8-bit if it's not already
run("8-bit");

// Set the threshold
setThreshold({min_val}, {max_val});

// Run the Analyze Particles command
run("Analyze Particles...", "size=0-Infinity clear summarize");

// Get the particle count from the results table
count = getResult("Count", 0);

// Return the particle count as a string
return "" + count;
"""

# Execute the macro using ij.py.run_macro
result = ij.py.run_macro(macro_code)

# Print the number of dots
print(f"Number of dots: {result}")

Jul 22, 2024 4:05:27 PM java.util.prefs.FileSystemPreferences$1 run
INFO: Created user preferences directory.
[ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage
Operating in headless mode - the original ImageJ will have limited functionality.


[INFO] script:macro.ijm = [[result], [NaN]]
Number of dots: org.scijava.script.ScriptModule@1aa4c4d3
